# Resumimos las Fotos de Previsión

De forma que tengamos la ocupación a una fecha para cada Hotel y Fecha Foto, así como el total de habitaciones del hotel, para poder calcular el % de Ocupación. De esta forma tendremso nuestos valores normalizados.

_Importación de librerias_

In [1]:
import pandas as pd

import tfm_helpers as tfm
pd.set_option("display.max_columns", None)

Procesamos los ficheros para generar previsiones agrupadas por año

In [2]:
years_to_process = ["2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019"]

resumen_ocupacion_data = pd.DataFrame()

for year in years_to_process:
    file = tfm.getPathToProcessedSnapshotOcupacion("parquet", year)

    current_ocupacion_data = pd.read_parquet(file, columns=["Hotel", "Fecha foto", "Fecha prevision", "Cantidad habitaciones"])
    current_ocupacion_data["Hotel"] = current_ocupacion_data.Hotel.astype("category")
    current_ocupacion_data.set_index(["Hotel", "Fecha foto", "Fecha prevision"], inplace=True)
    
    current_ocupacion_data = pd.DataFrame(current_ocupacion_data.groupby(["Hotel", "Fecha foto", "Fecha prevision"])["Cantidad habitaciones"].agg("sum"))
    
    filtro_habitaciones = (current_ocupacion_data["Cantidad habitaciones"]>0)
    resumen_ocupacion_data = pd.concat([resumen_ocupacion_data, current_ocupacion_data[filtro_habitaciones]])


In [3]:
resumen_ocupacion_data.head()

Cantidad habitaciones
Hotel Fecha foto Fecha prevision                       
04    2005-10-31 2006-03-24                           1
                 2006-03-25                           1
                 2006-03-26                           1
                 2006-03-27                           1
                 2006-03-28                           1

_Procesamos el resumen de habitaciones por hotel_

* Tipos de Habitación

In [4]:
tipos_habitacion_dtypes = {
    "Hotel" : str
}

tipos_habitacion_parse_dates = [
    "Fecha"
]

tipos_habitacion_cols = list(tipos_habitacion_dtypes.keys()) + tipos_habitacion_parse_dates + ["RAL Cantidad AA"]

In [5]:
all_hotels_tipos_habitacion = pd.DataFrame()
for hotel in tfm.hoteles:   
    files = tfm.getListOfFiles(tfm.getSourcePathToHistoricalPresupuestoTiposHabitacionHotel(tfm.data_source_path, hotel))
    for file in files:
        current_data = pd.read_csv(
                            file, 
                            sep="|", 
                            escapechar="~",
                            encoding="cp1252", 
                            on_bad_lines="skip", 
                            encoding_errors="ignore",
                            quoting=3,
                            usecols=tipos_habitacion_cols,
                            dtype=tipos_habitacion_dtypes, 
                            parse_dates=tipos_habitacion_parse_dates,
                            converters={"RAL Cantidad AA":tfm.to_integer}
                        )
        current_data["Hotel"] = current_data.Hotel.astype("category")
        
        current_data.set_index(["Hotel", "Fecha"], inplace=True)
        all_hotels_tipos_habitacion = pd.concat([all_hotels_tipos_habitacion, current_data])
        all_hotels_tipos_habitacion.sort_index(inplace=True)


In [6]:
all_hotels_tipos_habitacion.head()

RAL Cantidad AA
Hotel Fecha                      
02    2009-01-01                0
      2009-01-01                0
      2009-01-01                0
      2009-01-01                0
      2009-01-01                0

_Unificamos todos los tipos de habitación para obtener un total de habitaciones diario por Hotel_

In [7]:
all_hotels_tipos_habitacion = all_hotels_tipos_habitacion.groupby(["Hotel", "Fecha"]).sum()

_Hacemos un merge de ambos dataframes para poder realizar los cálculos del % de ocupación_

In [8]:
prevision_ocupacion = pd.merge(resumen_ocupacion_data, all_hotels_tipos_habitacion, how="left", left_on=["Hotel", "Fecha prevision"], right_index=True)
prevision_ocupacion.rename(columns={"RAL Cantidad AA":"Habitaciones Hotel"}, inplace=True)
prevision_ocupacion.sort_index(inplace=True)

In [9]:
prevision_ocupacion.head()

Cantidad habitaciones  Habitaciones Hotel
Hotel Fecha foto Fecha prevision                                           
02    2012-06-10 2013-09-19                           1               297.0
                 2013-09-20                           1               297.0
                 2013-09-21                           1               297.0
                 2013-09-22                           1               297.0
                 2013-09-23                           1               297.0

_Guardamos los resultados_

In [10]:

prevision_ocupacion.to_parquet(tfm.getPathToProcessedSnapshotResumenOcupacion("parquet"), index=True)